In [1]:
import numpy as np
import pandas as pd
import tushare as ts

In [2]:
ts.set_token('4c1d16a895e4c954adc8d2a436f2b21dd4ccc514f0c5a192edaa953b')
pro = ts.pro_api()
df = pro.index_weight(index_code='000905.SH', trade_date='20090128')
df = pro.index_weight(index_code='000905.SH', start_date='20090101', end_date='20090131')

In [3]:
from pymongo import MongoClient, ASCENDING, DESCENDING
mc = MongoClient('localhost', 27017)  # Mongo连接
db_stock_digu = mc['stock_digu']  # 数据库

In [4]:
class StockMongodb:
    def __init__(self, hostname='127.0.0.1', db_port=27017):

        """初始化类，初始化数据库连接"""

        self.hostname = hostname
        self.db_port = db_port
        self.my_mongo_client = MongoClient(self.hostname, self.db_port)

    def df2mongo(self, df_data, db_name, form_name):

        """DataFrame数据写入mongodb"""

        def df2bson(df):

            """DataFrame类型转化为Bson类型"""

            data = json.loads(df.T.to_json()).values()
            return data

        my_db = self.my_mongo_client[db_name]
        bson_data = df2bson(df_data)
        my_posts = my_db[form_name]
        result = my_posts.insert_many(bson_data)
        return result

    def collection2df(self, db_name, collection_name, query={}, no_id=True):

        """查询数据库，导出DataFrame类型数据
        （db_name：数据库名 collection_name：集合名 
         query：查询条件式 no_id：不显示ID,默认为不显示ID）"""

        db = self.my_mongo_client[db_name]
        collection = self.my_mongo_client[db_name][collection_name]
        cursor = collection.find(query)
        df = pd.DataFrame(list(cursor))
        if no_id:
            del df['_id']
        return df

In [5]:
df_index_sh = db_stock_digu['000001_SH'].find({}, {'_id': 0})

In [6]:
stock_mongodb = StockMongodb()
df_index_sh = stock_mongodb.collection2df('stock_digu', '000001_SH')

In [7]:
# 标签采用pct_chg字段，并对其分段处理
from sklearn.preprocessing import KBinsDiscretizer
pct_chg = df_index_sh.loc[:,'pct_chg'].values.reshape(-1, 1)
est = KBinsDiscretizer(n_bins=6, encode='ordinal', strategy='quantile')
res = est.fit_transform(pct_chg)
df_index_sh_y = res.ravel()[1:]

In [8]:
est.bin_edges_

array([array([-7.7245 , -0.87155, -0.2808 ,  0.08575,  0.4167 ,  0.9593 ,
        5.7113 ])], dtype=object)

In [9]:
df_index_sh_X = df_index_sh.iloc[:-1, ~df_index_sh.columns.isin(['pct_chg'])]

In [10]:
df_index_sh_X.loc[:, 'ma_5_'] = df_index_sh_X.loc[:, 'close'] - df_index_sh_X.loc[:, 'ma_5']

In [11]:
df_index_sh_X.head()
df_index_sh_X.loc[:,'ma_5_close'] = (df_index_sh_X.loc[:, 'ma_5'] - df_index_sh_X.loc[:, 'close']) / df_index_sh_X.loc[:, 'close']
df_index_sh_X.loc[:,'ma_10_close'] = (df_index_sh_X.loc[:, 'ma_10'] - df_index_sh_X.loc[:, 'close']) / df_index_sh_X.loc[:, 'close']
df_index_sh_X.loc[:,'ma_20_close'] = (df_index_sh_X.loc[:, 'ma_20'] - df_index_sh_X.loc[:, 'close']) / df_index_sh_X.loc[:, 'close']
df_index_sh_X.loc[:,'ma_30_close'] = (df_index_sh_X.loc[:, 'ma_30'] - df_index_sh_X.loc[:, 'close']) / df_index_sh_X.loc[:, 'close']
df_index_sh_X.loc[:,'ma_60_close'] = (df_index_sh_X.loc[:, 'ma_60'] - df_index_sh_X.loc[:, 'close']) / df_index_sh_X.loc[:, 'close']
df_index_sh_X.loc[:,'ma_120_close'] = (df_index_sh_X.loc[:, 'ma_120'] - df_index_sh_X.loc[:, 'close']) / df_index_sh_X.loc[:, 'close']
df_index_sh_X.loc[:,'ma_5_10'] = (df_index_sh_X.loc[:, 'ma_5'] - df_index_sh_X.loc[:, 'ma_10']) / df_index_sh_X.loc[:, 'ma_10']
df_index_sh_X.loc[:,'ma_10_20'] = (df_index_sh_X.loc[:, 'ma_10'] - df_index_sh_X.loc[:, 'ma_20']) / df_index_sh_X.loc[:, 'ma_20']
df_index_sh_X.loc[:,'ma_20_30'] = (df_index_sh_X.loc[:, 'ma_20'] - df_index_sh_X.loc[:, 'ma_30']) / df_index_sh_X.loc[:, 'ma_30']
df_index_sh_X.loc[:,'ma_30_60'] = (df_index_sh_X.loc[:, 'ma_30'] - df_index_sh_X.loc[:, 'ma_60']) / df_index_sh_X.loc[:, 'ma_60']
df_index_sh_X.loc[:,'ma_60_120'] = (df_index_sh_X.loc[:, 'ma_60'] - df_index_sh_X.loc[:, 'ma_120']) / df_index_sh_X.loc[:, 'ma_120']
df_X = df_index_sh_X.loc[:, ['ma_5_close', 'ma_10_close', 'ma_20_close', 'ma_30_close', 'ma_60_close', 'ma_120_close', 'ma_5_10', 'ma_10_20', 'ma_20_30', 'ma_30_60', 'ma_60_120']]
df_X = df_X.iloc[120:, :]
df_X

,ma_5_close,ma_10_close,ma_20_close,ma_30_close,ma_60_close,ma_120_close,ma_5_10,ma_10_20,ma_20_30,ma_30_60,ma_60_120
120,0.000230,-0.003889,-0.023301,-0.029701,-0.004167,-0.034770,0.004134,0.019875,0.006597,-0.025641,0.031705
121,0.004126,0.000034,-0.017653,-0.025209,-0.002187,-0.030054,0.004091,0.018005,0.007751,-0.023072,0.028730
122,0.004326,-0.001594,-0.016505,-0.025334,-0.005406,-0.030552,0.005929,0.015162,0.009059,-0.020037,0.025938
123,0.023322,0.022301,0.010929,0.001395,0.019197,-0.003737,0.000999,0.011249,0.009521,-0.017467,0.023020
124,0.017213,0.022255,0.012744,0.003558,0.019199,-0.000847,-0.004932,0.009391,0.009153,-0.015345,0.020063
125,0.014866,0.024696,0.017337,0.008070,0.022148,0.004644,-0.009593,0.007234,0.009192,-0.013773,0.017423
126,0.008014,0.021124,0.016600,0.007256,0.019739,0.004890,-0.012839,0.004450,0.009277,-0.012242,0.014777
127,-0.001879,0.015020,0.012993,0.003143,0.013884,0.001577,-0.016648,0.002000,0.009820,-0.010594,0.012288
128,-0.005228,0.007513,0.009913,-0.000333,0.008109,-0.001234,-0.012646,-0.002377,0.010250,-0.008374,0.009355
129,-0.003041,0.005460,0.012292,0.001761,0.007830,0.001361,-0.008456,-0.006748,0.010512,-0.006022,0.006460


In [12]:
df_y = df_index_sh_y[120:]
df_y

array([1., 3., 0., 2., 1., 2., 4., 3., 2., 2., 0., 4., 0., 4., 4., 4., 3.,
       2., 3., 1., 1., 0., 0., 0., 1., 4., 1., 5., 1., 4., 3., 3., 5., 2.,
       2., 3., 4., 0., 5., 1., 2., 2., 5., 3., 4., 5., 4., 4., 2., 1., 4.,
       2., 0., 3., 4., 3., 0., 3., 0., 0., 3., 0., 3., 3., 4., 4., 5., 1.,
       1., 2., 0., 2., 4., 1., 2., 4., 4., 1., 1., 1., 5., 4., 4., 1., 2.,
       1., 0., 3., 1., 3., 1., 4., 4., 1., 2., 1., 4., 2., 2., 1., 1., 3.,
       3., 2., 4., 4., 2., 3., 3., 1., 5., 4., 5., 2., 2., 1., 0., 4., 2.,
       4., 2., 5., 3., 5., 2., 2., 4., 0., 4., 2., 4., 2., 1., 1., 2., 4.,
       0., 3., 0., 0., 5., 5., 5., 3., 4., 3., 4., 1., 3., 5., 2., 1., 5.,
       3., 2., 1., 1., 3., 0., 5., 4., 4., 5., 0., 0., 5., 0., 0., 0., 0.,
       1., 0., 0., 5., 0., 5., 5., 1., 3., 0., 3., 1., 5., 1., 5., 2., 3.,
       0., 1., 5., 1., 3., 4., 4., 0., 4., 2., 0., 3., 2., 4., 5., 4., 2.,
       4., 2., 2., 3., 0., 2., 3., 4., 1., 1., 0., 3., 5., 1., 3., 3., 5.,
       3., 2., 2., 3., 3.

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

params = {'criterion': ['entropy', 'gini']
          ,'max_depth':np.arange(1, 20, 1)
          ,'min_samples_split':np.arange(2, 20, 1)
          ,'min_samples_leaf':np.arange(1, 10, 1)
         }
rfc = RandomForestClassifier(n_estimators=300, n_jobs=-1, oob_score=True)
gs = GridSearchCV(rfc, param_grid=params, cv=10)
gs.fit(df_X, df_y)

KeyboardInterrupt: 

In [17]:
rfc.get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [83]:
rfc.estimator_params

('criterion',
 'max_depth',
 'min_samples_split',
 'min_samples_leaf',
 'min_weight_fraction_leaf',
 'max_features',
 'max_leaf_nodes',
 'min_impurity_decrease',
 'min_impurity_split',
 'random_state',
 'ccp_alpha')

In [41]:
X_1 = X.loc[:, [2, 'aaa']]
X_1

,2,aaa
0,2,-1
1,6,-1
2,10,-1
3,14,-1


In [48]:
df_index_sh_X = df_index_sh.iloc[:-1, ~df_index_sh.columns.isin(['pct_chg'])]

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15


In [50]:
df1.head()

,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount
0,000905.SH,20200817,6744.5468,6646.5418,6752.3176,6614.1849,6623.5633,120.9835,1.8266,201407357.0,207540650.7
1,000905.SH,20200814,6623.5633,6542.6032,6629.5058,6506.0266,6552.6567,70.9066,1.0821,156875988.0,161300612.5
2,000905.SH,20200813,6552.6567,6552.3695,6592.1163,6518.4229,6528.3999,24.2568,0.3716,162060515.0,166812245.2
3,000905.SH,20200812,6528.3999,6580.0058,6599.6573,6395.0060,6600.8472,-72.4473,-1.0975,177889045.0,197969436.4
4,000905.SH,20200811,6600.8472,6727.7692,6756.8451,6594.9793,6725.2956,-124.4484,-1.8505,180845771.0,203427777.8


In [9]:
df1.tail()

,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount
1365,000905.SH,20150109,5409.058,5424.501,5497.689,5409.058,5444.037,-34.979,-0.6425,90640808.0,1.053903e+08
1366,000905.SH,20150108,5444.037,5482.394,5497.744,5418.011,5488.242,-44.205,-0.8054,95634617.0,1.085210e+08
1367,000905.SH,20150107,5488.242,5471.402,5492.825,5436.918,5479.864,8.378,0.1529,96050494.0,1.117871e+08
1368,000905.SH,20150106,5479.864,5389.903,5482.124,5354.338,5417.017,62.847,1.1602,123754496.0,1.403281e+08
1369,000905.SH,20150105,5417.017,5338.752,5427.231,5314.339,5322.714,94.303,1.7717,128787603.0,1.393430e+08


In [15]:
import numpy
import talib

close = np.array([1.0,2,3,4,5])

In [55]:
talib.SMA(close, 5)

array([nan, nan, nan, nan,  3.])

In [46]:
df1.loc[:,'trade_date'] = '111'

In [47]:
df1

,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount
0,000905.SH,111,6744.5468,6646.5418,6752.3176,6614.1849,6623.5633,120.9835,1.8266,201407357.0,2.075407e+08
1,000905.SH,111,6623.5633,6542.6032,6629.5058,6506.0266,6552.6567,70.9066,1.0821,156875988.0,1.613006e+08
2,000905.SH,111,6552.6567,6552.3695,6592.1163,6518.4229,6528.3999,24.2568,0.3716,162060515.0,1.668122e+08
3,000905.SH,111,6528.3999,6580.0058,6599.6573,6395.0060,6600.8472,-72.4473,-1.0975,177889045.0,1.979694e+08
4,000905.SH,111,6600.8472,6727.7692,6756.8451,6594.9793,6725.2956,-124.4484,-1.8505,180845771.0,2.034278e+08
5,000905.SH,111,6725.2956,6666.5601,6768.8660,6651.7421,6688.8822,36.4134,0.5444,175353227.0,2.103909e+08
6,000905.SH,111,6688.8822,6743.2907,6755.5280,6578.6672,6770.3016,-81.4194,-1.2026,198128214.0,2.279767e+08
7,000905.SH,111,6770.3016,6780.3265,6792.2759,6656.2782,6772.8422,-2.5406,-0.0375,205229842.0,2.409289e+08
8,000905.SH,111,6772.8422,6696.6870,6790.0080,6653.2772,6697.7071,75.1351,1.1218,184843816.0,2.179942e+08
9,000905.SH,111,6697.7071,6758.4937,6764.9049,6672.8940,6739.8085,-42.1014,-0.6247,209478936.0,2.450848e+08


In [79]:
pd.Series([1,1,1,1,1])

0    1
1    1
2    1
3    1
4    1
dtype: int64

In [82]:
a=  pd.Series([1,1,1,1,1])
dft.loc[:, 'tt'] = a

In [83]:
dft

,index_code,trade_date,weight,ts_code,tt
0,000905.SH,20180928,0.261,603899_SH,1
1,000905.SH,20180928,0.108,603888_SH,1
2,000905.SH,20180928,0.218,603885_SH,1
3,000905.SH,20180928,0.163,603883_SH,1
4,000905.SH,20180928,0.037,603877_SH,1


In [17]:
df = pro.index_dailybasic(ts_code='000905.SH', start_date='20150101', end_date='20200218')

In [18]:
df.shape

(1247, 12)

In [19]:
df.head()

,ts_code,trade_date,total_mv,float_mv,total_share,float_share,free_share,turnover_rate,turnover_rate_f,pe,pe_ttm,pb
0,000905.SH,20200218,9.277410e+12,7.690620e+12,1.050439e+12,8.851686e+11,4.876428e+11,2.09,3.80,25.10,26.40,1.95
1,000905.SH,20200217,9.176565e+12,7.606636e+12,1.050439e+12,8.851686e+11,4.875427e+11,1.97,3.58,24.83,26.12,1.92
2,000905.SH,20200214,8.900520e+12,7.373632e+12,1.050439e+12,8.849777e+11,4.872903e+11,1.64,2.98,24.08,25.33,1.87
3,000905.SH,20200213,8.891418e+12,7.361154e+12,1.050387e+12,8.849250e+11,4.871510e+11,1.79,3.25,24.06,25.30,1.86
4,000905.SH,20200212,8.966612e+12,7.422433e+12,1.050387e+12,8.842708e+11,4.871431e+11,1.71,3.10,24.26,25.52,1.88


In [20]:
df.columns

Index(['ts_code', 'trade_date', 'total_mv', 'float_mv', 'total_share',
       'float_share', 'free_share', 'turnover_rate', 'turnover_rate_f', 'pe',
       'pe_ttm', 'pb'],
      dtype='object')

In [5]:
from vnpy.app.cta_strategy.backtesting import BacktestingEngine, OptimizationSetting
# from vnpy.app.cta_strategy.strategies.atr_rsi_strategy import AtrRsiStrategy
from trade_stock_digu.strategy.strategy_zz500 import ZZ500Strategy
from datetime import datetime

engine = BacktestingEngine()
engine.set_parameters(
    vt_symbol="000905_SH.SSE",
    interval="d",
    start=datetime(2013, 1, 1),
    end=datetime(2019, 12, 31),
    rate=0.1/10000,
    slippage=0.1,
    size=300,
    pricetick=0.2,
    capital=1_000_000,
)
engine.add_strategy(ZZ500Strategy, {})

engine.load_data()
engine.run_backtesting()
print(ZZ500Strategy.c1)
print(ZZ500Strategy.c2)
print(ZZ500Strategy.c3)
print(ZZ500Strategy.c4)
print(ZZ500Strategy.c5)
print(ZZ500Strategy.c6)
df = engine.calculate_result()
engine.calculate_statistics()
engine.show_chart()

setting = OptimizationSetting()
setting.set_target("sharpe_ratio")
setting.add_parameter("atr_length", 3, 39, 1)
setting.add_parameter("atr_ma_length", 10, 30, 1)

# engine.run_ga_optimization(setting)


2020-08-27 01:59:23.547807	开始加载历史数据
2020-08-27 01:59:23.548805	加载进度： [1%]
2020-08-27 01:59:23.548805	加载进度： [2%]
2020-08-27 01:59:23.548805	加载进度： [4%]
2020-08-27 01:59:23.548805	加载进度： [5%]
2020-08-27 01:59:23.548805	加载进度： [6%]
2020-08-27 01:59:23.548805	加载进度： [7%]
2020-08-27 01:59:23.548805	加载进度： [8%]
2020-08-27 01:59:23.548805	加载进度： [9%]
2020-08-27 01:59:23.548805	加载进度：# [11%]
2020-08-27 01:59:23.548805	加载进度：# [12%]
2020-08-27 01:59:23.548805	加载进度：# [13%]
2020-08-27 01:59:23.548805	加载进度：# [14%]
2020-08-27 01:59:23.548805	加载进度：# [15%]
2020-08-27 01:59:23.548805	加载进度：# [16%]
2020-08-27 01:59:23.548805	加载进度：# [18%]
2020-08-27 01:59:23.548805	加载进度：# [19%]
2020-08-27 01:59:23.548805	加载进度：# [20%]
2020-08-27 01:59:23.548805	加载进度：## [21%]
2020-08-27 01:59:23.548805	加载进度：## [22%]
2020-08-27 01:59:23.548805	加载进度：## [23%]
2020-08-27 01:59:23.548805	加载进度：## [25%]
2020-08-27 01:59:23.548805	加载进度：## [26%]
2020-08-27 01:59:23.548805	加载进度：## [27%]
2020-08-27 01:59:23.548805	加载进度：## [28%]
2020-08-27 01

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('dingzengjiejin2020.csv')

In [13]:
from trade_stock_digu.data_service import DataServiceTushare

In [6]:
df.head()

,code,date_free
0,002352.SZ,20200123
1,300441.SZ,20200423
2,300603.SZ,20200203
3,300647.SZ,20200506
4,603501.SH,20200831


In [17]:
df1 = df[df['date_free'] < 20200801]

In [19]:
# 计算定增解禁股解禁日前后20个交易日信息
# 解禁日之前的20日作为基准日：计算收盘最大涨幅（跌幅）平均值和中位数
ds_tushare = DataServiceTushare()
lst_up = list()
lst_down = list()
for index, row in df1.iterrows():
    date_end = ds_tushare.get_next_trade_date(str(row["date_free"]), 20)
    date_start = ds_tushare.get_pre_trade_date(str(row["date_free"]), 20)
    lst_close = list()
    lst_k_data = ds_tushare.get_stock_price_lst(row["code"], date_start, date_end)
    for item in lst_k_data:
        lst_close.append(item['close'])
    if len(lst_close) == 0:
        continue
    price_base = lst_close[0]
    pct_up = (max(lst_close) - price_base) / price_base
    pct_down = (min(lst_close) - price_base) / price_base
    lst_up.append(pct_up)
    lst_down.append(pct_down)
mean_up = np.mean(lst_up)
mean_down = np.mean(lst_down)
median_up = np.median(lst_up)
median_down = np.median(lst_down)

c:\vnstudio\lib\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\vnstudio\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [20]:
print(mean_up, median_up, mean_down, median_down)

nan nan nan nan


In [22]:
lst_down

[]